# Translation between Excel & CSV

### Convert Excel Sheets into CSV for RStudio Usage

##### Read in Worksheets

In [1]:
import numpy
import pandas as pd
from openpyxl import load_workbook
from vbaFunc import jumpoff, xTFR, clear_Input
from vbaFunc import write_Accounts, write_SmallAreaInputs, write_Log, write_CheckMig, write_CheckDeaths, write_AgeSexForecasts, write_Components, write_Target

# Load Workbooks
wt_loc = 'vba.xlsx'
wb_wt = load_workbook(wt_loc)

# Read in Sheets
df_labels = pd.read_excel('original.xlsm', sheet_name='Labels')
df_AgeSex = pd.read_excel('original.xlsm', sheet_name='AgeSexERPs')
df_AreaTotal = pd.read_excel('original.xlsm', sheet_name='SmallAreaTotals')
df_Fertility = pd.read_excel('original.xlsm', sheet_name='Fertility')
df_Mortality = pd.read_excel('original.xlsm', sheet_name='Mortality')
df_Migration = pd.read_excel('original.xlsm', sheet_name='Migration')
df_NationalProjection = pd.read_excel('original.xlsm', sheet_name='NationalProjection')

# Load Worksheets (Write & Read)
wb_wt_SmallAreaInputs = wb_wt["SmallAreaInputs"]
wb_wt_Accounts = wb_wt["Accounts"]
wb_wt_fertility = wb_wt["Fertility"]
wb_wt_AgeSexForecasts = wb_wt["AgeSexForecasts"]
wb_wt_Components = wb_wt["Components"]
wb_wt_Target = wb_wt["Target"]
wb_wt_CheckMig = wb_wt["CheckMig"]
wb_wt_CheckDeaths = wb_wt["CheckDeaths"]
wb_wt_Check_OK = wb_wt["Check_OK"]
wb_wt_Log = wb_wt["Log"]

##### Convert Worksheets into CSV(s)

In [2]:
# Define Variables
areas = df_labels.iloc[2255, 0]
final = df_labels.iloc[2270, 0]
ages = 18

# Labels
# Labels Sheet Information
labels = df_labels.iloc[3:(3+areas), 0:3]
labels.columns = ["No.", "Code", "Area name"]
labels.to_csv("../VBA_9616/Input_CSV/labels.csv", index = False)

# Age / Period Groups Information
age_group = pd.DataFrame(df_labels.iloc[3:3+ages, 4:6])
period_cohort = pd.DataFrame(df_labels.iloc[3:3+ages, 7:9])
age_cohorts = pd.concat([age_group, period_cohort], axis = 1, join = 'inner')
age_cohorts.columns = ["Projection age groups", "age_value", "Period-cohorts", "period_value"]
age_cohorts.to_csv("../VBA_9616/Input_CSV/age_cohorts.csv", index = False)

# Other Key Information from Labels Sheets
year_labels = (df_labels.iloc[2258, 3:16]).astype(int)
interval_labels = df_labels.iloc[2261, 3:15]
jump_off = [(df_labels.iloc[3, 10])] + (len(year_labels) - 1) * [numpy.nan]
maxqiter = [(df_labels.iloc[9, 10])] + (len(year_labels) - 1) * [numpy.nan]
maxziter = [(df_labels.iloc[12, 10])] + (len(year_labels) - 1) * [numpy.nan]
final_list = [(final)] + (len(year_labels) - 1) * [numpy.nan]
areas_list = [(areas)] + (len(year_labels) - 1) * [numpy.nan]
interval_labels = interval_labels.tolist() + [numpy.nan]
labels_info = pd.DataFrame(year_labels)
labels_info["Period-cohorts"] = interval_labels
labels_info["Jump-off year"] = jump_off
labels_info["Maxqiter"] = maxqiter
labels_info["Maxziter"] = maxziter
labels_info["final"] = final_list
labels_info["No. of small areas"] = areas_list
labels_info = labels_info.rename(columns = {labels_info.columns[0]: "Year labels"})
labels_info.to_csv("../VBA_9616/Input_CSV/labels_other_key.csv", index = False)

##########################################################################################################################################################################
# AgeSexERPs
df_AgeSexERPs = df_AgeSex.iloc[3:(3+areas*ages), 0:8]
df_AgeSexERPs.columns = ["No.", "Code", "Name", "Age", "Males" + str(int(df_AgeSex.iloc[2, 4])), "Males" + str(int(df_AgeSex.iloc[2, 5])), \
                        "Females" + str(int(df_AgeSex.iloc[2, 6])), "Females" + str(int(df_AgeSex.iloc[2, 7]))]
df_AgeSexERPs.to_csv("../VBA_9616/Input_CSV/AgeSexERPs.csv", index = False)

##########################################################################################################################################################################
# SmallAreaTotals
df_SmallAreaTotal = df_AreaTotal.iloc[3:3+areas, ].dropna(axis=1, how='all')
proj_years = len(df_SmallAreaTotal.columns) - 3
name_list = ["No.", "Code", "Name"]
for i in range(proj_years):
    name_list.append(str(int(df_AreaTotal.iloc[2, 3 + i])))
df_SmallAreaTotal.columns = name_list
df_SmallAreaTotal.to_csv("../VBA_9616/Input_CSV/SmallAreaTotals.csv", index = False)

##########################################################################################################################################################################
# Other Fertility Information
## Age-specific Fertility Rates
ASFR_prelim = df_Fertility.iloc[5:5+areas, 16:16+7+1].fillna(0)
ASFR_prelim.columns = df_Fertility.iloc[2, 16:16+7+1].tolist()
ASFR_prelim.to_csv("../VBA_9616/Input_CSV/ASFR_Prelim.csv", index = False)

## Other Key Data
ASFR_values = df_Fertility.iloc[2258:2265, 1].tolist()
TFR = [sum(ASFR_values) * 5] + [numpy.nan] * (len(ASFR_values) - 1)
SRB = [df_Fertility.iloc[2269, 1]] + [numpy.nan] * (len(ASFR_values) - 1)
otherFertility = pd.DataFrame(ASFR_values)
otherFertility["TFR"] = TFR
otherFertility["SRB"] = SRB
otherFertility = otherFertility.rename(columns = {otherFertility.columns[0]: "ASFR"})
otherFertility.to_csv("../VBA_9616/Input_CSV/otherFertility.csv", index = False)

##########################################################################################################################################################################
# Mortality Information
## Life Expectancy
Expectancy = df_Mortality.iloc[2:5+areas,].dropna(axis=1, how='all').dropna(axis=0, how='all').dropna(axis=1, how='any')
Expectancy = pd.DataFrame(Expectancy).iloc[0:areas+1, 1:]
interval_list = Expectancy.iloc[0, 0:].tolist()
sex_list = ["Females"] * int(len(interval_list) / 2) + ["Males"] * int(len(interval_list) / 2)
for i in range(len(interval_list)):
    interval_list[i] = sex_list[i] + "_" + interval_list[i]
Expectancy = pd.DataFrame(Expectancy).iloc[1:areas+1, ]
Expectancy.columns = interval_list
Expectancy.to_csv("../VBA_9616/Input_CSV/LifeExpectancy.csv", index = False)
## nLx Mortaility Surface
Mortality = df_Mortality.iloc[2259:, 3:26].dropna(axis=0, how='any')
nLx_name = ["sex", "age"]
for i in range(1, 22):
    nLx_name.append(str(i))
Mortality.columns = nLx_name
Mortality.to_csv("../VBA_9616/Input_CSV/Mortality.csv", index = False)

##########################################################################################################################################################################
# Migration Information
## Base Period Migration Turnover
column_name = df_Migration.iloc[2, 0:5].tolist()
Migration_Turnover = df_Migration.iloc[4:4+areas, 0:5]
Migration_Turnover.columns = column_name
## Crude Migration Turnover Rate
Crude_turnover = [df_Migration.iloc[5, 7]] + [numpy.nan] * (areas - 1)
Migration_Turnover["Crude"] = Crude_turnover
Migration_Turnover.to_csv("../VBA_9616/Input_CSV/migTurnover.csv", index = False)
## Model Migration Rates
migRate = df_Migration.iloc[10:10+ages, 6:9]
migRate.columns = ["period_value", "Females", "Males"]
migRate.to_csv("../VBA_9616/Input_CSV/migRate.csv", index = False)

##########################################################################################################################################################################
# NationalProjection Information
## Total Population
TotPopulation = df_NationalProjection.iloc[1:3+2*ages, ].dropna(axis=0, how='all').dropna(axis=1, how='any')
column_name = TotPopulation.iloc[0, ].tolist()
TotPopulation = TotPopulation.iloc[1:, ]
TotPopulation.columns = column_name
TotPopulation.to_csv("../VBA_9616/Input_CSV/TotPopulation.csv", index = False)
## Total Birth
TotBirth = df_NationalProjection.iloc[41:42+2, ].dropna(axis=0, how='all').dropna(axis=1, how='any')
column_name = TotBirth.iloc[0, ].tolist()
TotBirth = TotBirth.iloc[1:]
TotBirth.columns = column_name
TotBirth["Sex"] = ["Females", "Males"]
TotBirth.insert(0, 'Sex', TotBirth.pop('Sex'))
TotBirth.to_csv("../VBA_9616/Input_CSV/totBirth.csv", index = False)
## Total Death
TotDeath = df_NationalProjection.iloc[47:47+2*ages, 1:6]
column_name = df_NationalProjection.iloc[46:48, ].dropna(axis=1, how='any')
column_name = column_name.iloc[0, ].tolist()
column_name = ["Sex", "period_value"] + column_name
TotDeath.columns = column_name
TotDeath.to_csv("../VBA_9616/Input_CSV/totDeath.csv", index = False)
## Tot Net Migration
TotMig = df_NationalProjection.iloc[86:86+2*ages, 1:6]
column_name = df_NationalProjection.iloc[85:87, ].dropna(axis=1, how='any')
column_name = column_name.iloc[0, ].tolist()
column_name = ["Sex", "period_value"] + column_name
TotMig.columns = column_name
TotMig.to_csv("../VBA_9616/Input_CSV/totNetMig.csv", index = False)

# Finish Excel -> CSV Convertion
print("Convertion Finished")

Convertion Finished


##### Special Write to Excel (Fertility)

In [ ]:
# Fertility (Special Data) -- Repeat Generation in Python for Inserting Data to Excel
# Same Calculation is Presented in RStudio as well
# Read in Excel Workbook (both Readable and Writable)
rd_loc = 'original.xlsm'
wb_rd = load_workbook(rd_loc, data_only = True)
fert_years = df_Fertility.iloc[2, 3:3+final+1].tolist()

# Special: Fertility Data
# Read in Sheets
sheet_label = wb_rd["Labels"]
sheet_agesex = wb_rd["AgeSexERPs"]

# Define variables
numareas = int(sheet_label.cell(2257, 1).value)
numages = 18
sextypes = 2
set_year_female = 8
set_year_male = 6

# Create empty array for storing jumpoffERP and xTFR results
jumpoffERP = numpy.zeros((numareas, sextypes, numages))
jumpoffERP[:] = numpy.nan
result_xTFR = numpy.zeros((numareas))
result_xTFR[:] = numpy.nan

# Generate jumpoffERP for further calculation
jumpoffERP = jumpoff(numareas, numages, jumpoffERP, sheet_agesex, set_year_female, set_year_male)

# Generate xTFR vector for storing
result_xTFR = xTFR(jumpoffERP, result_xTFR, numareas)

# Store into Sheet 'Fertility' for further usage
row = 7
column = 4
for i in range(len(result_xTFR)):
    wb_wt_fertility.cell(row, column).value = result_xTFR[i]
    row += 1
wb_wt.save(wt_loc)

##### Clear Previous Input & Output

In [ ]:
# Before running the PrepareData Function, clear the input data if any
wb_wt_SmallAreaInputs = wb_wt["SmallAreaInputs"]
wb_wt_Accounts = wb_wt["Accounts"]
wb_wt_AgeSexForecasts = wb_wt["AgeSexForecasts"]
wb_wt_Components = wb_wt["Components"]
wb_wt_Target = wb_wt["Target"]
wb_wt_CheckMig = wb_wt["CheckMig"]
wb_wt_CheckDeaths = wb_wt["CheckDeaths"]
wb_wt_Check_OK = wb_wt["Check_OK"]
wb_wt_Log = wb_wt["Log"]
wb_wt_Target = wb_wt["Target"]

# Clear the data and save the writable workbook
clear_Input(wb_wt_SmallAreaInputs)
clear_Input(wb_wt_Accounts)
clear_Input(wb_wt_AgeSexForecasts)
clear_Input(wb_wt_Components)
clear_Input(wb_wt_CheckMig)
clear_Input(wb_wt_CheckDeaths)
clear_Input(wb_wt_Check_OK)
clear_Input(wb_wt_Log)
clear_Input(wb_wt_Target)
wb_wt.save(wt_loc)

### Convert CSV to Excel for Delivering Results

In [ ]:
# Read in CSV Files
Accounts = pd.read_csv("../VBA_9616/Output_CSV/Accounts.csv")
SmallAreaInputs = pd.read_csv("../VBA_9616/Output_CSV/SmallAreaInputs.csv")
Log = pd.read_csv("../VBA_9616/Output_CSV/Log.csv")
CheckMig = pd.read_csv("../VBA_9616/Output_CSV/CheckMig.csv")
CheckDeaths = pd.read_csv("../VBA_9616/Output_CSV/CheckDeaths.csv")
AgeSexForecasts = pd.read_csv("../VBA_9616/Output_CSV/AgeSexForecasts.csv")
Components = pd.read_csv("../VBA_9616/Output_CSV/Components.csv")

# Write CSV into Worksheet
wb_wt_Accounts = write_Accounts(Accounts, wb_wt_Accounts, numages)
wb_wt_SmallAreaInputs = write_SmallAreaInputs(SmallAreaInputs, wb_wt_SmallAreaInputs, numages, numareas, final)
wb_wt_Log = write_Log(Log, wb_wt_Log, final)
wb_wt_CheckMig = write_CheckMig(CheckMig, wb_wt_CheckMig, numareas, numages, final)
wb_wt_CheckDeaths = write_CheckDeaths(CheckDeaths, wb_wt_CheckDeaths, numareas, numages, final)
wb_wt_AgeSexForecasts = write_AgeSexForecasts(AgeSexForecasts, wb_wt_AgeSexForecasts)
wb_wt_Components = write_Components(Components, wb_wt_Components, numareas, final)
wb_wt.save(wt_loc)

### Projection Visualisation

In [ ]:
# Visualisation of Summary of Population Projection with Selected Area and Projection Year
# Set male's data as left section and female's data as right section in the Pyramid Plot
Target_Area = "Queanbeyan"
Jump_Year = 2020
Proj_Year = 2035

# Write target data into sheet
wb_wt_Target = write_Target(wb_wt_Target, Target_Area, Jump_Year, Proj_Year, numages, AgeSexForecasts)
wb_wt.save(wt_loc)